In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html, callback_context, Input, Output

PLAYING = 1
PAUSED  = 2
STOPPED = 3

app = JupyterDash()
player: object = None

def timer_input():
    v = player.value
    title = 'Play'
    disabled = True
    if v == player.maximum:
        player.state = PAUSED
    if player.state == PLAYING:
        if v is None:
            v = player.minimum
        else:
            v += 1
        title = 'Pause'
        disabled = False
    elif player.state == PAUSED:
        pass
    else: #stopped
        v = None
    return v, title, disabled

def play_button_input():
    v = player.value
    title = 'Play'
    disabled = True
    if player.state == PLAYING:
        player.state = PAUSED
    else:
        if v == player.maximum:
            v = None
        title = 'Pause'
        disabled = False
        player.state = PLAYING
    return v, title, disabled

def stop_button_input():
    v = None
    title = 'Play'
    disabled = True
    player.state = STOPPED
    return v, title, disabled

@app.callback(
    Output('slider-output-container', 'children'),
    Input('my-slider', 'value'))
def update_output(value):
    if value == player.value:
        return
    player.executeCallback(value)
    return

@app.callback([Output('my-slider', 'value'),
               Output('play-button', 'children'),
               Output('timer', 'disabled')],
              [Input('timer', 'n_intervals'),
               Input('play-button', 'n_clicks'),
               Input('stop-button', 'n_clicks')])
def update_metrics(n_timer, n_play, n_stop):
    if not callback_context.triggered:
        return None, 'Play', True
    trigger = callback_context.triggered[0]['prop_id'].split('.')[0]
    if trigger == 'timer':
        return timer_input()
    elif trigger == 'play-button':
        return play_button_input()
    elif trigger == 'stop-button':
        return stop_button_input()
    else:
        return None, 'Play', True

class DataPlayer(object):
    def __init__(self, minimum=1, maximum=1, interval=50, callback=None, mode='inline', port=8050, width='100%', height='650px'):
        self.app = app
        self.mode = mode
        self.port = port
        self.width = width
        self.height = height
        self.minimum = minimum
        self.maximum = maximum
        self.interval = interval
        self.callback = callback
        self.value = None
        self.state = STOPPED
        global player
        player = self
        self.slider = dcc.Slider(min=minimum, max=maximum, step=1, value=None, id='my-slider')
        self.app.layout = html.Div([
            html.Div([self.slider]),
            html.Div([html.Button('Play', id='play-button', n_clicks=0, style={'width':'50px', 'height':'30px', 'cursor':'pointer'}),
                      html.Button('Stop', id='stop-button', n_clicks=0, style={'width':'50px', 'height':'30px', 'cursor':'pointer'})]),
            dcc.Interval(id='timer', interval=interval),
            html.Div(id='slider-output-container')
        ])

    def run(self):
        self.app.run_server(debug=True, mode=self.mode, port=self.port, width=self.width, height=self.height)

    def executeCallback(self, value):
        self.value = value
        if value is None:
            return
        if self.callback is not None:
            self.callback(value)
